In [ ]:
import pandas as pd
import numpy as np
from tsa.preprocessing import get_sample_info, tpm_normalization
from tsa.gpr import gpr
from tsa.gene_selection import score_normalization, plot_scores, best_n_genes
from tsa.tsa import get_cost_matrix, best_alignment_graph, plot_alignment
from tsa.utils import list2floats, inference_timeseries
from tsa.plotting import plot_alignments

In [ ]:
# samples = pd.read_csv("/bank/experiments/2020-07/s2s_batch_corr2/samples.tsv", sep="\t", index_col=0)
# tpms = pd.read_csv("/bank/experiments/2020-09/s2s/gene_counts/Xenopus_tropicalis_v9.1-TPM.tsv", sep="\t", index_col=0)

In [ ]:
# samples.batch.unique()

In [ ]:
# GPR input files
tpm_file = "data/GRCz11-TPM.tsv"
# template_samples_file = "data/white_stage_samples.tsv"
template_samples_file = "data/white_mpf_samples.tsv"

# variables
timepoints_per_sample = 10

# GPR output files
# gpr_inference_file = "data/white_stage_gpr.tsv"
# gpr_scores_file = "data/white_stage_score.tsv"
gpr_inference_file = "data/white_mpf_gpr.tsv"
gpr_scores_file = "data/white_mpf_score.tsv"

# gene selection
# n_genes = 500
selected_genes_file = "data/white_genes.tsv"

# GPR

In [ ]:
# preprocessing
template_samples = pd.read_csv(template_samples_file, sep="\t", index_col=0)
sample_order, time2samples = get_sample_info(template_samples)

tpms = pd.read_csv(tpm_file, sep="\t", index_col=0)
template_tpms = tpm_normalization(tpms, sample_order, minimum_value=5)

# filter genes BEFORE running GPR (optional)
selected_genes = list(pd.read_csv(selected_genes_file, sep="\t")["gene"])
template_tpms = template_tpms[template_tpms.index.isin(selected_genes)]

# GPR (slow)

In [ ]:
# linear space
extended_timepoints = list(np.round(np.linspace(min(time2samples), max(time2samples), 513), 2))

# n points per input
# extended_timepoints = inference_timeseries(list(time2samples), timepoints_per_sample)

print(len(extended_timepoints))
extended_timepoints

In [ ]:
template_tpms_inf, gpr_scores = gpr(time2samples, template_tpms, extended_timepoints, plot=False, verbose=False, run_n=None)
# template_tpms_inf.to_csv(gpr_inference_file, sep="\t")
# gpr_scores.to_csv(gpr_scores_file, sep="\t")

# # gene selection (optional)
# gpr_normscores = score_normalization(gpr_scores)
# plot_scores(gpr_normscores, highlight_top_n=n_genes)
# # save selected genes
# selected_genes = best_n_genes(gpr_normscores, n_genes=n_genes, to_file=selected_genes_file)

template_tpms_inf

# TSA

In [ ]:
# TSA input files
# tpm_file = "data/GRCz11-TPM.tsv"
# template_samples_file = "data/white_stage_samples.tsv"
# template_samples_file = "data/white_mpf_samples.tsv"
# gpr_inference_file = "data/white_stage_gpr.tsv"
# selected_genes_file = "data/white_stage_selected_genes.tsv"

# query_samples_file = "data/white_stage_samples.tsv"
# query_samples_file = "data/levin_stage_samples.tsv"
# query_samples_file = "data/marletaz_stage_samples.tsv"
# query_samples_file = "data/white_mpf_samples.tsv"
# query_samples_file = "data/levin_mpf_samples.tsv"
query_samples_file = "data/marletaz_mpf_samples.tsv"

# TSA output files
# alignment_file = "data/white_stage_white_stage_mapping.tsv"
# alignment_file = "data/white_stage_levin_stage_mapping.tsv"
# alignment_file = "data/white_stage_marletaz_stage_mapping.tsv"
# alignment_file = "data/white_mpf_white_mpf_mapping.tsv"
# alignment_file = "data/white_mpf_levin_mpf_mapping.tsv"
# alignment_file = "data/white_mpf_marletaz_mpf_mapping.tsv"
# alignment_file = "data/white_mpf_linear_white_mpf_mapping.tsv"
# alignment_file = "data/white_mpf_linear_levin_mpf_mapping.tsv"
alignment_file = "data/white_mpf_linear_marletaz_mpf_mapping.tsv"

In [ ]:
# preprocessing
query_samples = pd.read_csv(query_samples_file, sep="\t", index_col=0)
sample_order, time2samples = get_sample_info(query_samples)

tpms = pd.read_csv(tpm_file, sep="\t", index_col=0)
query_tpms = tpm_normalization(tpms, sample_order)

# cost matrix
# selected_genes = list(pd.read_csv(selected_genes_file, sep="\t")["gene"])
# template_tpms_inf = pd.read_csv(gpr_inference_file, sep="\t", index_col=0)
cost_matrix = get_cost_matrix(template_tpms_inf, query_tpms, selected_genes, time2samples)

# LTSA
best_path, best_score = best_alignment_graph(cost_matrix)
plot_alignment(cost_matrix, best_path)

# mapping
query_time = list2floats(query_samples.time.unique())
extended_template_time = list2floats(template_tpms_inf.columns)
mapped = pd.DataFrame(data={
    "original_time": query_time,
    "inferred_time": [extended_template_time[i] for i in best_path],
})
mapped.to_csv(alignment_file, sep="\t", index=False)  # noqa

# Timeline

In [ ]:
# Timeline input files
# template_samples_file = "data/white_stage_samples.tsv"
# template_samples_file = "data/white_mpf_samples.tsv"
# gpr_inference_file = "data/white_mpf_gpr.tsv"

alignment_files = {
    "white": "data/white_stage_white_stage_mapping.tsv",
    "levin": "data/white_stage_levin_stage_mapping.tsv",
    "marletaz": "data/white_stage_marletaz_stage_mapping.tsv",
}
# alignment_files = {
#     "white": "data/white_mpf_white_mpf_mapping.tsv",
#     "levin": "data/white_mpf_levin_mpf_mapping.tsv",
#     "marletaz": "data/white_mpf_marletaz_mpf_mapping.tsv",
# }
# alignment_files = {
#     "white": "data/white_mpf_1000_white_mpf_mapping.tsv",
#     "levin": "data/white_mpf_1000_levin_mpf_mapping.tsv",
#     "marletaz": "data/white_mpf_1000_marletaz_mpf_mapping.tsv",
# }

####################################################

# x axis
# template_samples = pd.read_csv(template_samples_file, sep="\t", index_col=0)
# template_tpms_inf = pd.read_csv(gpr_inference_file, sep="\t", index_col=0)

template_time = list2floats(template_samples.time.unique())
extended_template_time = list2floats(template_tpms_inf.columns)
# extended_timepoints = inference_timeseries(template_time, timepoints_per_sample)
plot_alignments(template_time, extended_template_time, alignment_files)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tsa.utils import inference_timeseries


# universal zebrafish time axis (from ZFIN)
all_mpf = [
    0,
    45,
    60,
    75,
    90,
    105,
    120,
    135,
    150,
    165,
    180,
    200,
    220,
    240,
    260,
    280,
    315,
    340,
    360,
    480,
    540,
    600,
    620,
    700,
    840,
    960,
    1140,
    1320,
    1440,
    1800,
    2160,
    2520,
    2880,
    3600,
    4320,
    5760,
    7200,
    8640,
    10080,
    20160,
    30240,
    43200,
    64800,
][:-6]
all_stages = [
    "1-cell",
    "2-cell",
    "4-cell",
    "8-cell",
    "16-cell",
    "32-cell",
    "64-cell",
    "128-cell",
    "256-cell",
    "512-cell",
    "1k-cell",
    "High",
    "Oblong",
    "Sphere",
    "Dome",
    "30%-epiboly",
    "50%-epiboly",
    "Germ-ring",
    "Shield",
    "75%-epiboly",
    "90%-epiboly",
    "Bud",
    "1-4 somites",
    "5-9 somites",
    "10-13 somites",
    "14-19 somites",
    "20-25 somites",
    "26+ somites",
    "Prim-5",
    "Prim-15",
    "Prim-25",
    "High-pec",
    "Long-pec",
    "Pec-fin",
    "Protruding-mouth",
    "Day 4",
    "Day 5",
    "Day 6",
    "Days 7-13",
    "Days 14-20",
    "Days 21-29",
    "Days 30-44",
    "Days 45-89",
][:-6]


#########################################


# query alignments
# alignment_files = {
#     "white": "data/white_stage_white_stage_mapping.tsv",
#     "levin": "data/white_stage_levin_stage_mapping.tsv",
#     "marletaz": "data/white_stage_marletaz_stage_mapping.tsv",
# }
alignment_files = {
    "white": "data/white_mpf_white_mpf_mapping.tsv",
    "levin": "data/white_mpf_levin_mpf_mapping.tsv",
    "marletaz": "data/white_mpf_marletaz_mpf_mapping.tsv",
}
# alignment_files = {
#     "white": "data/white_mpf_linear_white_mpf_mapping.tsv",
#     "levin": "data/white_mpf_linear_levin_mpf_mapping.tsv",
#     "marletaz": "data/white_mpf_linear_marletaz_mpf_mapping.tsv",
# }

# output plot
plot_file = "data/alignment_white_mpf.pdf"

plt.rcParams['figure.figsize'] = [24, 8]
fig = plt.figure(1)
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()

n = 0
ylabels = []
for series in alignment_files:
    df = pd.read_csv(alignment_files[series], sep="\t")
    x1 = df["original_time"]
    x2 = df["inferred_time"]
    plt.scatter(x=x1, y=np.zeros_like(x1) + n)
    n += 1
    plt.scatter(x=x2, y=np.zeros_like(x2) + n)
    n += 1
    ylabels.extend([f"{series} annotated", f"{series} inferred"])

# plot shape
x_axis = all_mpf
x_range = max(x_axis) - min(x_axis)
plt.yticks(list(range(n)), ylabels)
plt.ylim(-0.5, n - 0.5)
plt.ylabel("time series")

total_time = max(x_axis) - min(x_axis)
plt.xlim(min(x_axis) - x_range * 0.03, max(x_axis) + x_range * 0.03)

ax1.set_xticks(x_axis)
ax1.set_xticklabels(all_stages, rotation=45, ha="right")
ax1.set_xlabel("developmental stage")

ax2.set_xticks(x_axis)
ax2.set_xticklabels(all_mpf, rotation=45, ha="left")
ax2.set_xlabel("minutes post fertilization")

plt.savefig(plot_file)
plt.show()


#########################################


# # template time
# template_samples_file = "data/white_stage_samples.tsv"
# timepoints_per_sample = 10

# # query alignments
# alignments = {
#     "white": "data/white_stage_white_stage_mapping.tsv",
#     "levin": "data/white_stage_levin_stage_mapping.tsv",
#     "marletaz": "data/white_stage_marletaz_stage_mapping.tsv",
# }

# # output plot
# plot_file = "data/alignment_white_stage.pdf"



# template_samples = pd.read_csv(template_samples_file, sep="\t", index_col=0)
# template_time = list(template_samples.time.unique())
# extended_template_time = inference_timeseries(template_time, timepoints_per_sample)

# ext_time_axis = pd.DataFrame({
#     "ext_all_mpf": inference_timeseries(all_mpf, timepoints_per_sample),
#     "ext_all_stages": inference_timeseries(all_stages, timepoints_per_sample),
# })

# plt.rcParams['figure.figsize'] = [24, 8]
# fig = plt.figure(1)
# ax1 = fig.add_subplot(111)
# ax2 = ax1.twiny()

# n = 0
# ylabels = []
# for series in alignments:
#     df = pd.read_csv(alignments[series], sep="\t")

#     x1 = df["original_time"]
#     if df["original_time"].dtype == "O":
#         cv = ext_time_axis.merge(df, left_on="ext_all_stages", right_on="original_time", how="right")
#         x1 = cv["ext_all_mpf"].to_list()
#     plt.scatter(x=x1, y=np.zeros_like(x1) + n)
#     n += 1

#     x2 = df["inferred_time"]
#     if df["inferred_time"].dtype == "O":
#         cv = ext_time_axis.merge(df, left_on="ext_all_stages", right_on="inferred_time", how="right")
#         x2 = cv["ext_all_mpf"].to_list()
#     plt.scatter(x=x2, y=np.zeros_like(x2) + n)
#     n += 1
#     ylabels.extend([f"{series} annotated", f"{series} inferred"])

# # plot shape
# x_axis = all_mpf[:-2]
# x_labels1 = all_stages[:-2]
# x_labels2 = all_mpf[:-2]
# x_range = max(x_axis) - min(x_axis)
# plt.yticks(list(range(n)), ylabels)
# plt.ylim(-0.5, n - 0.5)
# plt.ylabel("time series")

# # total_time = max(x_axis) - min(x_axis)
# # plt.xlim(min(x_axis), max(x_axis))
# plt.xlim(min(x_axis) - x_range * 0.03, max(x_axis) + x_range * 0.03)
# # plt.ylim(-0.5, 2.5)

# ax1.set_xticks(x_axis)
# ax1.set_xticklabels(x_labels1, rotation=45, ha="right")
# ax1.set_xlabel("developmental stage")

# ax2.set_xticks(x_axis)
# ax2.set_xticklabels(x_labels2, rotation=45, ha="left")
# ax2.set_xlabel("minutes post fertilization")

# plt.savefig(plot_file)
# plt.show()